# IT Academy - Data Science amb Python
## Tasca 7: Estructura de control

###  [Github Registre de Logs](https://github.com/jesussantana/Registre_de_logs)

###  Exercici 1
- Normalitza, identifica i enumera cada un dels atributs / variables de l'estructura de l'arxiu "Web_access_log-akumenius.com" que trobaràs al repositori de GitHub "Data-sources".

In [ ]:
import numpy as np
import pandas as pd
import time
import requests
import re
from datetime import datetime
from datetime import timedelta
import collections, numpy

pd.set_option('display.max_columns', None)

In [ ]:
path = '../Data/Web_access_log-akumenius.com.txt'

Logs_raw = pd.read_csv(path, sep='\s | \- | \"', names =['DNS','IP','Location','Date','Time','Request','Status','Size','Referer','UserAgent'], engine='python')

Logs_copy = Logs_raw.copy()

Logs_copy.head()

###  Exercici 2
- Neteja, preprocesa, estructura i transforma (dataframe) les dades del registre d'Accés a la web.

In [ ]:
Logs_copy.tail()

In [ ]:
Logs_copy.shape

In [ ]:
Logs_copy.isnull().sum()

In [ ]:
Logs_copy.head()

In [ ]:
Logs_copy.tail()

In [ ]:
# Reorder columns of data

Logs_copy.UserAgent = Logs_copy.Time
Logs_copy.Request = Logs_copy.Location
Logs_copy.Referer = Logs_copy.Date
Logs_copy.Date = Logs_copy.IP
Logs_copy.Time = Logs_copy.IP = Logs_copy.Location = np.nan

Logs_copy.head()

In [ ]:
# DNS & IP data split

Logs_copy[['DNS','IP']] = Logs_copy.DNS.str.split('\s', expand = True).get([0, 1])

In [ ]:
Logs_copy.head()

In [ ]:
Logs_copy.DNS.unique()

In [ ]:
len(Logs_copy.IP.unique())

In [ ]:
# Time data extraction

Logs_copy.Time = Logs_copy.Date.str.extract(':(\d{2}:\d{2}:\d{2}.*)]')

#Logs_copy.Time = pd.to_datetime(Logs_copy.Time, format = '%H:%M:%S %z')
Logs_copy.head()

In [ ]:
# Date data extraction

Logs_copy.Date = Logs_copy.Date.str.extract('(\d+/\w+/\d+)')

#Logs_copy.Date = pd.to_datetime(Logs_copy.Date, format = '%d/%b/%Y')

In [ ]:
Logs_copy.head()

In [ ]:
# Request & Status data split

Logs_copy[['Request','Status']] = Logs_copy.Request.str.split('\"', expand = True).get([0, 1])

In [ ]:
Logs_copy.tail()

In [ ]:
Logs_copy.head()

In [ ]:
# Size data extraction

Logs_copy.Size = Logs_copy.Status.str.extract('(\d+$)')
Logs_copy.head()

In [ ]:
# Clean Size data
start = time.monotonic()

sizes = []

for index, row in Logs_copy.iterrows(): 
    
    #if re.search('200', row.Size):
    if row.Size == '200':
        sizes.append(np.nan)
    else:
        sizes.append(row.Size)

Logs_copy.Size = sizes

end = time.monotonic()

print(timedelta(seconds = end - start))

Logs_copy.head()

In [ ]:
# Status data extraction

Logs_copy.Status = Logs_copy.Status.str.extract('(\d{3})')
Logs_copy.tail()

In [ ]:
# Clean Referer data

start = time.monotonic()

referers = []

for index, row in Logs_copy.iterrows(): 
    
    if re.search('-"', row.Referer):
        referers.append(np.nan)
    else:
        referers.append(row.Referer.rstrip(row.Referer[-1]))

    

Logs_copy.Referer = referers

In [ ]:
Logs_copy

###  Exercici 3
- Geolocalitza les IP's. Aqui tens una pagina de interes:
  - [IP2Locattion](https://blog.ip2location.com/knowledge-base/how-to-add-a-sub-account-in-ip2location/)

In [ ]:
Ips_export= Logs_copy.IP.copy()

Ips_export.replace('', 'null', inplace = True)

Ips_export.to_csv('../Data/Ips_export.csv', index = False)

- Extracion de Localizaciones 
  - [Extract_Locattion](https://github.com/jesussantana/Registre_de_logs/Tasca_7_Registre_de_Logs_Ips_Exercici3.ypynb)

In [ ]:
'''import json
from ip2geotools.databases.noncommercial import DbIpCity'''

In [ ]:
# Extract Ips

'''ips = Logs_copy.IP.unique()'''

In [ ]:
# Function for extract Locations

'''localhost = "127.0.0.1"

def extract_location(ip):
    
    try:
        if ip == localhost:
            return ('Local', 'Local')

        else:
            response = DbIpCity.get(ip, api_key = 'free').to_json()    
            data = json.loads(response)
            return data['city'], data['country']
    
    except:
        return np.nan'

In [ ]:
# List of locations for unique Ips

'''start = time.monotonic()

locations = []

for item in ips:
    
    locations.append(extract_location(item))  

end = time.monotonic()

print(timedelta(seconds = end - start))'''

In [ ]:
# Extract Location Test

'''extract_location(Logs_copy.IP[1000])

In [ ]:
# Compare ips & add location

'''start = time.monotonic()

IP_locations = []

for index, row in Logs_copy.iterrows():
    
    # iterate the two lists
    for a, b in zip(ips, locations):
        
        # Compare & add
        if row.IP == a:
            IP_locations.append(b)  

end = time.monotonic()

print(timedelta(seconds = end - start))'''

In [ ]:
'''Logs_copy.Location = IP_locations

In [ ]:
path = '../Data/Locations_export.csv'

Locations_raw = pd.read_csv(path, sep='delimiter', names =['IP','Location'], engine='python')

Locations_copy = Locations_raw.copy()

Locations_copy.head()

In [ ]:
Logs_copy'''

In [ ]:
Locations_copy.tail()

In [ ]:
Logs_copy.Location = Locations_copy

In [ ]:
Logs_copy.head()

In [ ]:
Logs_copy.tail()

###  Exercici 4
- Mostreu-me la teva creativitat, Sorprèn-me fes un pas més enllà amb el analysis anterior.

In [ ]:
# Extract UserAgent Data

from device_detector import SoftwareDetector

Devices = Logs_copy.UserAgent
device = []

for x in Devices:
    
    device.append(SoftwareDetector(x).parse())

client_name = []
client_type = []
client_version = []
os_name = []
os_version = []
device_type = []

for x in device:
    
    client_name.append(x.client_name())
    client_type.append(x.client_type())
    client_version.append(x.client_version())
    os_name.append(x.os_name())
    os_version.append(x.os_version())
    device_type.append(x.device_type())

Logs_copy['Client_Name'] = client_name
Logs_copy['Client_Type'] = client_type
Logs_copy['Client_Version'] = client_version
Logs_copy['Os_Name'] = os_name
Logs_copy['Os_Version'] = os_version
Logs_copy['Device_Type'] = device_type

Logs_copy

In [ ]:
Logs_copy.UserAgent[0]

In [ ]:
del(Logs_copy['UserAgent'])

Logs_copy.tail()

In [ ]:
# Clean Client Data
client = []

for index, row in Logs_copy.iterrows(): 
    
    if re.search('Apache', row.Client_Name):
        client.append(row.Client_Name[0:6])
    else:
        client.append(row.Client_Name)

Logs_copy.Client_Name = client

Logs_copy.head()

In [ ]:
Logs_copy

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [ ]:

Logs_copy.Location.value_counts(ascending = True).plot(kind = 'barh', figsize = (10,6))

In [ ]:
graph = Logs_copy[['DNS', 'Location']].groupby(['DNS']).count().sort_values(by = 'Location',ascending = False)
graph = graph.rename(columns = {'Location' : 'Frequency'})
graph.plot.bar(y = 'Frequency', color = 'g', ylabel = 'Frequency', legend = None, figsize = (10,6))
plt.show()

In [ ]:
graph = Logs_copy[['Status', 'Time']].groupby(['Status']).count().sort_values(by = 'Time',ascending = False)
graph = graph.rename(columns = {'Time' : 'Frequency'})
graph.plot.bar(y = 'Frequency', color = 'r', ylabel = 'Frequency', legend = None, figsize = (10,6))
plt.show()

In [ ]:
graph = Logs_copy[['Client_Type', 'Time']].groupby(['Client_Type']).count().sort_values(by = 'Time',ascending = False)
graph = graph.rename(columns = {'Time' : 'Frequency'})
graph.plot.bar(y = 'Frequency', color = 'y', ylabel = 'Frequency', legend = None, figsize = (10,6))
plt.show()

In [ ]:
sns.displot(data = Logs_copy, x = "Os_Name", hue = "Date", multiple = "stack")

In [ ]:
sns.displot(data = Logs_copy, x = "Device_Type", hue = "Date", multiple = "stack")

In [ ]:
Logs_export = Logs_copy.copy()

Logs_export.replace('', 'null', inplace = True)

Logs_export.to_csv('../Data/Logs_export.csv', index = False)